In [39]:
import pandas as pd 
from cadCAD.configuration.utils import config_sim

In [40]:
# Additional dependencies

# For parsing the data from the API
import json
# For downloading data from API
import requests as req
# For generating random numbers
import math
# For visualization
import plotly.express as px
# For Google BigQuery authentication
from google.oauth2 import service_account

In [5]:
%%capture

credentials = service_account.Credentials.from_service_account_file(
    './credentials.json',
)

QUERY = """
SELECT * FROM blockchain-etl.ethereum_balancer.V2_Vault_event_PoolBalanceChanged
WHERE poolId = "0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f8000200000000000000000019"
ORDER BY block_timestamp DESC
"""

# Send the SQL query to the ethereum-etl dataset
# on Google BigQuery.
# Requires the pandas-gbq library
supply_data = pd.read_gbq(QUERY, project_id="blockchain-319713", credentials=credentials)

# Print the last 5 rows
supply_data.tail(5)

In [21]:
supply_data

,block_timestamp,block_number,transaction_hash,log_index,contract_address,poolId,liquidityProvider,tokens,deltas,protocolFeeAmounts
0,2021-07-19 11:26:02+00:00,12856721,0xbb2ebb05b3bf77b2fc4f4c8d590d74aeb6aa9cd7beb3...,264,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0x8c64e3cf9248443d0fee48e695297044ddf64c01,"0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xC...","-6915050448,-3704995746946645349","0,0"
1,2021-07-19 10:52:02+00:00,12856584,0x0936ff9eceac32a3f6a14b7dd393ca99fcadc06910fd...,239,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0x87428db7c48aad07e2b565756b2e70064973104a,"0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xC...","-2234944376666,-1195010956012635280761","0,0"
2,2021-07-19 05:48:13+00:00,12855229,0x66ff7a5a1dfeb450025ca4708dfb191649aacad10f57...,225,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0x1a35500310b0afffa170bca27cdc9f8a4453d0fb,"0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xC...","0,6158584200264247682","0,0"
3,2021-07-18 20:31:39+00:00,12852773,0x78c136ec0d96fb6e31cd6d891b1c2b1209d9bc29c1ab...,3,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0x89039e9fe8905ff0a0e83cd36a21f49a6feabfdd,"0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xC...","0,-397316941285266894","0,0"
4,2021-07-18 11:29:25+00:00,12850342,0xad382d508de1474c47bd0ecac3833e1eecd6ec067097...,316,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xabf727d85b90503dcc175f97a4674f649f369540,"0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xC...","0,-1106014660550850229","0,0"
...,...,...,...,...,...,...,...,...,...,...
494,2021-05-14 04:59:58+00:00,12430516,0xf9bbde4ca8b15f55fbcaf27b784ce588da002605f219...,277,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0x1b7a9c831b4f2088fa6eae337088ae2fc9913ef5,"0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xC...","103278260,0","0,0"
495,2021-05-13 16:43:35+00:00,12427174,0xc8a1d669e134e1d9ce48c2e6c8a3eb5d0563b78af3a0...,249,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0x42d13efa958586ff2d3ec20956d44dad8da55864,"0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xC...","225504472441,58570000000000000000","0,0"
496,2021-05-13 16:32:37+00:00,12427130,0x1897c1c456f06c913b64164728cb4447a57e4644fac5...,131,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xdf290293c4a4d6ebe38fd7085d7721041f927e0a,"0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xC...","180535524958,47100000000000000000","0,0"
497,2021-05-12 22:02:56+00:00,12422142,0x68fc92874ec821f77553e3ad113b9b7324c756d3a6f7...,38,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0x65b43ee5d4421652b34f85ad057a93ba0a43b866,"0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xC...","166192077,0","0,0"


In [7]:
%%capture

credentials = service_account.Credentials.from_service_account_file(
    './credentials.json',
)

QUERY = """
SELECT * FROM blockchain-etl.ethereum_balancer.V2_Vault_event_Swap
WHERE poolId = "0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f8000200000000000000000019"
ORDER BY block_timestamp DESC
"""

# Send the SQL query to the ethereum-etl dataset
# on Google BigQuery.
# Requires the pandas-gbq library
swap_data = pd.read_gbq(QUERY, project_id="blockchain-319713", credentials=credentials)

# Print the last 5 rows
swap_data.tail(5)

In [ ]:
tokens = {
    "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48": {
        "ticker": "UDSC",
        "decimal": 6
    },
    "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2": {
        "ticker": "WETH",
        "decimal": 18
    }
}

In [25]:
swap_data

,block_timestamp,block_number,transaction_hash,log_index,contract_address,poolId,tokenIn,tokenOut,amountIn,amountOut
0,2021-07-19 15:15:04+00:00,12857785,0xd4c5d0c2244f024ac5dc2aef182340a16afe66be21c4...,150,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10000000000,5463595512559581918
1,2021-07-19 15:11:13+00:00,12857773,0x0dfaffd9e588ca92b2ecb971c795a373639b978326c8...,127,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,19968809018,10927000000000000000
2,2021-07-19 14:52:15+00:00,12857685,0x01d1c312959da67850e8d70ca0bb066082d617527edf...,76,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10000000000,5480476737891348938
3,2021-07-19 14:48:47+00:00,12857673,0x192c9ba8eb7d64c3c691e69fda2c2b419490dc12b8bc...,88,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,19982116485,10968100000000000000
4,2021-07-19 13:59:06+00:00,12857406,0x0852d05719f25670dd34376c15e964d7f7f4c3960937...,64,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,15309931821276279715,27647471791
...,...,...,...,...,...,...,...,...,...,...
5282,2021-05-13 14:01:19+00:00,12426434,0x3b7ce67886af13c456ba4aba362ebb9ba28e96b8fa73...,91,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,63368661,17363384025566942
5283,2021-05-13 10:14:22+00:00,12425443,0x7c35b2702bb21e90f05a1e873fe7386c70e672501986...,30,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,33120883015159391,132253091
5284,2021-05-13 09:59:12+00:00,12425369,0x6e12b1594f7b2b44a703b0b39c8be5038d3955c7936e...,158,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,48000000,10299334491148577
5285,2021-05-12 22:13:43+00:00,12422185,0x8a96028fa6c510f4ca51d9b229901902fa36fc5f52e2...,225,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,9912500000000000,45336491


In [26]:
swap_data[]

,block_timestamp,block_number,transaction_hash,log_index,contract_address,poolId,tokenIn,tokenOut,amountIn,amountOut
0,2021-07-19 15:15:04+00:00,12857785,0xd4c5d0c2244f024ac5dc2aef182340a16afe66be21c4...,150,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10000000000,5463595512559581918
1,2021-07-19 15:11:13+00:00,12857773,0x0dfaffd9e588ca92b2ecb971c795a373639b978326c8...,127,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,19968809018,10927000000000000000
2,2021-07-19 14:52:15+00:00,12857685,0x01d1c312959da67850e8d70ca0bb066082d617527edf...,76,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10000000000,5480476737891348938
3,2021-07-19 14:48:47+00:00,12857673,0x192c9ba8eb7d64c3c691e69fda2c2b419490dc12b8bc...,88,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,19982116485,10968100000000000000
4,2021-07-19 13:59:06+00:00,12857406,0x0852d05719f25670dd34376c15e964d7f7f4c3960937...,64,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,15309931821276279715,27647471791
...,...,...,...,...,...,...,...,...,...,...
5282,2021-05-13 14:01:19+00:00,12426434,0x3b7ce67886af13c456ba4aba362ebb9ba28e96b8fa73...,91,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,63368661,17363384025566942
5283,2021-05-13 10:14:22+00:00,12425443,0x7c35b2702bb21e90f05a1e873fe7386c70e672501986...,30,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,33120883015159391,132253091
5284,2021-05-13 09:59:12+00:00,12425369,0x6e12b1594f7b2b44a703b0b39c8be5038d3955c7936e...,158,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,48000000,10299334491148577
5285,2021-05-12 22:13:43+00:00,12422185,0x8a96028fa6c510f4ca51d9b229901902fa36fc5f52e2...,225,0xba12222222228d8ba445958a75a0704d566bf2c8,0x96646936b91d6b9d7d0c47c496afbf3d6ec7b6f80002...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,9912500000000000,45336491


### Initialize pool

In [41]:
from decimal import *
from BalancerV2cad.WeightedPool import WeightedPool

In [42]:
wp = WeightedPool()
wp._swap_fee = Decimal(0.003)
wp.join_pool({'a':100,'b':100},{'a':0.5,'b':0.5})

print(wp.swap('b', 'a', 1, given_in=False))
print(wp._balances['a'], wp._balances['b'], wp.factory_fees)

1.013131313131313131376212257
99 101.0131313131313131313762123 {'a': Decimal('0'), 'b': Decimal('0.003030303030303030366111156972')}


In [43]:
from BalancerV2cad.WeightedMath import WeightedMath

In [44]:
# calc_in_given_out(balance_in: Decimal,
#                           weight_in: Decimal,
#                           balance_out: Decimal,
#                           weight_out: Decimal,
#                           amount_out: Decimal):

In [45]:
def changePoolWeights(wp, new_weights):
    new_pool = WeightedPool()
    balances = wp._balances
    new_pool.join_pool(balances,new_weights)
    new_pool.factory_fees = wp.factory_fees
    return new_pool

updated_wp = changePoolWeights(wp, {'a': 0.2, 'b': 0.8})

In [47]:
updated_wp._weights

{'a': Decimal('0.200000000000000011102230246251565404236316680908203125'),
 'b': Decimal('0.8000000000000000444089209850062616169452667236328125')}

In [46]:
updated_wp.factory_fees

{'a': Decimal('0'), 'b': Decimal('0.003030303030303030366111156972')}

In [30]:
from scipy.optimize import fsolve
import numpy as np
def func(x):
    return [x[0] * np.cos(x[1]) - 4,
            x[1] * x[0] - x[1] - 5]
root = fsolve(func, [1, 1])
print(root)

[6.50409711 0.90841421]


In [37]:
from scipy.optimize import fsolve
import numpy as np

a_bal = 100
a_weight = 0.5
b_bal = 100
b_weight = 0.5
new_price = 4.34739457389

def func(x):
    return [x[0] + x[1] - 1,
            a_bal*x[1] - new_price*b_bal*x[0]]
root = fsolve(func, [0.5, 0.5])
print(root)

[0.18700696 0.81299304]


In [36]:
# Stop when difference between weight-adjusted price is within 0.05% of new_price (< 0.0005x)
def find_optimal_weights(a_bal, a_weight, b_bal, b_weight, new_price):
    def func(x):
        return [x[0] + x[1] - 1,
            b_bal*x[0] - new_price*a_bal*x[1]]
    root = fsolve(func, [0.5, 0.5])
    return root



find_optimal_weights(100, 0.5, 100, 0.5, 4)

array([0.8, 0.2])